# Intro

Deze notebook toont de Python-implementatie van het protocol energiedelen versie 3.1

Het protocol beschrijft 3 verdeelsleutels: Vast, Relatief en Optimaal.

In [1]:
import pandas as pd

In [2]:
from openenergyid import TimeDataFrame
from openenergyid.energysharing import (
    EnergySharingInput,
    EnergySharingOutput,
    calculate,
    CalculationMethod,
    KeyInput,
)

## Input Data

Om de werking aan te tonen, maken we een sample dataset met 6 deelnemers, gedurende 4 kwartieren.

Elke deelnemer heeft een Bruto Afname en een Bruto Injectie. Voor elk kwartier is er een verdeelsleutel, die bepaalt hoeveel energie er naar elke deelnemer gaat. We wijzen een verdeelsleutel per kwartier to, omdat deze verdeelsleutels in de tijd kunnen veranderen.

Het resultaat zal zijn dat elke deelnemer een Netto Afname, een Netto Injectie en een hoeveelheid gedeelde energie heeft.

In [3]:
index = pd.date_range(
    start=pd.Timestamp("2024-01-01 12:00:00", tz="Europe/Brussels"),
    end=pd.Timestamp("2024-01-01 12:45:00", tz="Europe/Brussels"),
    freq="15min",
)

In [4]:
gross_injection = pd.DataFrame(
    data={
        "P1": [70, 20, 100, 0],
        "P2": [10, 10, 10, 10],
        "P3": [20, 30, 40, 50],
    },
    index=index,
)

gross_offtake = pd.DataFrame(
    data={
        "P2": [30, 20, 10, 0],
        "P3": [22, 20, 30, 40],
        "P4": [30, 25, 35, 10],
        "P5": [18, 17, 19, 25],
        "P6": [20, 22, 24, 26],
    },
    index=index,
)

key = pd.DataFrame(
    data={
        "P2": [10, 10, 10, 10],
        "P3": [22.5, 22.5, 22.5, 22.5],
        "P4": [22.5, 22.5, 22.5, 22.5],
        "P5": [22.5, 22.5, 22.5, 22.5],
        "P6": [22.5, 22.5, 22.5, 22.5],
    },
    index=index,
)
key = key.div(100)

In [5]:
gross_injection

,P1,P2,P3
2024-01-01 12:00:00+01:00,70,10,20
2024-01-01 12:15:00+01:00,20,10,30
2024-01-01 12:30:00+01:00,100,10,40
2024-01-01 12:45:00+01:00,0,10,50


In [6]:
gross_offtake

,P2,P3,P4,P5,P6
2024-01-01 12:00:00+01:00,30,22,30,18,20
2024-01-01 12:15:00+01:00,20,20,25,17,22
2024-01-01 12:30:00+01:00,10,30,35,19,24
2024-01-01 12:45:00+01:00,0,40,10,25,26


In [7]:
key

,P2,P3,P4,P5,P6
2024-01-01 12:00:00+01:00,0.1,0.225,0.225,0.225,0.225
2024-01-01 12:15:00+01:00,0.1,0.225,0.225,0.225,0.225
2024-01-01 12:30:00+01:00,0.1,0.225,0.225,0.225,0.225
2024-01-01 12:45:00+01:00,0.1,0.225,0.225,0.225,0.225


In [8]:
# Construct the input model
# This helps with validation and serialization

input = EnergySharingInput.model_construct(
    gross_injection=TimeDataFrame.from_pandas(gross_injection),
    gross_offtake=TimeDataFrame.from_pandas(gross_offtake),
    key=KeyInput.from_pandas(key),
)

with open("data/energiedelen/sample_input.json", "w") as f:
    f.write(input.model_dump_json(by_alias=True))

df = input.to_pandas()

In [9]:
df

Gross Injection             Gross Offtake        \
                                       P1    P2    P3            P2    P3   
2024-01-01 11:00:00+00:00            70.0  10.0  20.0          30.0  22.0   
2024-01-01 11:15:00+00:00            20.0  10.0  30.0          20.0  20.0   
2024-01-01 11:30:00+00:00           100.0  10.0  40.0          10.0  30.0   
2024-01-01 11:45:00+00:00             0.0  10.0  50.0           0.0  40.0   

                                             Key                              
                             P4    P5    P6   P2     P3     P4     P5     P6  
2024-01-01 11:00:00+00:00  30.0  18.0  20.0  0.1  0.225  0.225  0.225  0.225  
2024-01-01 11:15:00+00:00  25.0  17.0  22.0  0.1  0.225  0.225  0.225  0.225  
2024-01-01 11:30:00+00:00  35.0  19.0  24.0  0.1  0.225  0.225  0.225  0.225  
2024-01-01 11:45:00+00:00  10.0  25.0  26.0  0.1  0.225  0.225  0.225  0.225

# Vaste methode

![Methodiek vaste methode](data/energiedelen/Vast.png)

In [10]:
calculate(df, method=CalculationMethod.FIXED)

Net Injection                      Net Offtake  \
                                     P1        P2         P3          P2   
2024-01-01 11:00:00+00:00           4.9  1.700000   5.900000        21.0   
2024-01-01 11:15:00+00:00           0.0  1.000000   6.750000        15.0   
2024-01-01 11:30:00+00:00          19.0  2.900000  16.600000         0.0   
2024-01-01 11:45:00+00:00           0.0  2.416667  18.333333         0.0   

                                                    Shared Energy         \
                              P3     P4    P5    P6            P2     P3   
2024-01-01 11:00:00+00:00   4.00   7.50   0.0   0.0           9.0  18.00   
2024-01-01 11:15:00+00:00  13.25  11.50   3.5   8.5           5.0   6.75   
2024-01-01 11:30:00+00:00   5.25   1.25   0.0   0.0          10.0  24.75   
2024-01-01 11:45:00+00:00  37.75   0.00  11.5  12.5           0.0   2.25   

                                              
                              P4    P5    P6  
2024-01-01 11:00:00+00:00  22.50  18.0  20.0  
2024-01-01 11:15:00+00:00  13.50  13.5  13.5  
2024-01-01 11:30:00+00:00  33.75  19.0  24.0  
2024-01-01 11:45:00+00:00  10.00  13.5  13.5

# Relatieve methode

![Methodiek relatieve methode](data/energiedelen/Relatief.png)

In [11]:
calculate(df, method=CalculationMethod.RELATIVE)

Net Injection                         Net Offtake  \
                                     P1        P2            P3          P2   
2024-01-01 11:00:00+00:00      7.079032  1.011290  2.022581e+00   20.419355   
2024-01-01 11:15:00+00:00      0.000000  0.000000 -3.552714e-15   14.129032   
2024-01-01 11:30:00+00:00     24.666667  2.466667  9.866667e+00    0.000000   
2024-01-01 11:45:00+00:00      0.000000  2.244624  1.122312e+01    0.000000   

                                                               Shared Energy  \
                              P3        P4        P5        P6            P2   
2024-01-01 11:00:00+00:00   3.75  5.943548  0.000000  0.000000      9.580645   
2024-01-01 11:15:00+00:00  13.00  9.290323  1.290323  6.290323      5.870968   
2024-01-01 11:30:00+00:00   5.00  0.000000  0.000000  0.000000     10.000000   
2024-01-01 11:45:00+00:00  37.50  0.000000  7.983871  8.983871      0.000000   

                                                                   
                              P3         P4         P5         P6  
2024-01-01 11:00:00+00:00  18.25  24.056452  18.000000  20.000000  
2024-01-01 11:15:00+00:00   7.00  15.709677  15.709677  15.709677  
2024-01-01 11:30:00+00:00  25.00  35.000000  19.000000  24.000000  
2024-01-01 11:45:00+00:00   2.50  10.000000  17.016129  17.016129

# Optimale methode

## Iteratie 1

![Methodiek optimale methode iteratie 1](data/energiedelen/Optimaal_1.png)

## Iteratie 2

![Methodiek optimale methode iteratie 2](data/energiedelen/Optimaal_2.png)

In [12]:
calculate(df, method=CalculationMethod.OPTIMAL)

Net Injection                      Net Offtake  \
                                     P1        P2         P3          P2   
2024-01-01 11:00:00+00:00      0.000000  0.000000   0.000000   18.509926   
2024-01-01 11:15:00+00:00      0.000000  0.000000   0.000000   14.129032   
2024-01-01 11:30:00+00:00     14.755556  1.475556  15.768889    0.000000   
2024-01-01 11:45:00+00:00      0.000000  0.000000   0.000000    0.000000   

                                                                    \
                                  P3        P4        P5        P6   
2024-01-01 11:00:00+00:00   0.348387  1.141687  0.000000  0.000000   
2024-01-01 11:15:00+00:00  13.000000  9.290323  1.290323  6.290323   
2024-01-01 11:30:00+00:00   0.000000  0.000000  0.000000  0.000000   
2024-01-01 11:45:00+00:00  36.751792  0.000000  1.624104  2.624104   

                          Shared Energy                                   \
                                     P2         P3         P4         P5   
2024-01-01 11:00:00+00:00     11.490074  21.651613  28.858313  18.000000   
2024-01-01 11:15:00+00:00      5.870968   7.000000  15.709677  15.709677   
2024-01-01 11:30:00+00:00     10.000000  30.000000  35.000000  19.000000   
2024-01-01 11:45:00+00:00      0.000000   3.248208  10.000000  23.375896   

                                      
                                  P6  
2024-01-01 11:00:00+00:00  20.000000  
2024-01-01 11:15:00+00:00  15.709677  
2024-01-01 11:30:00+00:00  24.000000  
2024-01-01 11:45:00+00:00  23.375896

# Output Data

In [13]:
result = calculate(df, method=CalculationMethod.OPTIMAL)

In [14]:
output = EnergySharingOutput.from_calculation_result(result)

In [15]:
print(output.model_dump_json(by_alias=True, indent=2))

{
  "netInjection": {
    "index": [
      "2024-01-01T11:00:00Z",
      "2024-01-01T11:15:00Z",
      "2024-01-01T11:30:00Z",
      "2024-01-01T11:45:00Z"
    ],
    "columns": [
      "P1",
      "P2",
      "P3"
    ],
    "data": [
      [
        0.0,
        0.0,
        0.0
      ],
      [
        0.0,
        0.0,
        0.0
      ],
      [
        14.755555555555548,
        1.4755555555555548,
        15.768888888888883
      ],
      [
        0.0,
        0.0,
        0.0
      ]
    ]
  },
  "netOfftake": {
    "index": [
      "2024-01-01T11:00:00Z",
      "2024-01-01T11:15:00Z",
      "2024-01-01T11:30:00Z",
      "2024-01-01T11:45:00Z"
    ],
    "columns": [
      "P2",
      "P3",
      "P4",
      "P5",
      "P6"
    ],
    "data": [
      [
        18.50992555831266,
        0.3483870967741951,
        1.1416873449131553,
        0.0,
        0.0
      ],
      [
        14.129032258064516,
        13.0,
        9.290322580645162,
        1.2903225806451613,
   